<a href="https://colab.research.google.com/github/gopal2812/convandgpt/blob/main/Submission6/Session6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Install required packages
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import required packages
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision import datasets, transforms
from tqdm import tqdm

In [ ]:
# Class to define the NN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # # Reference Links
        # https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html
        # https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
        # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        # https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
        # https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
        # https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        # https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html
        # https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html
        # https://pytorch.org/docs/stable/generated/torch.Tensor.view.html

        # Using Sequential API to define the model as it seems to be more readable

        # General Notes

        # ReLU used after every Convolution layer
        # Batch Normalization used after every Convolution layer
        # Dropout used after every block/ layer
        # Max Pooling preferably used after every block
        # GAP used at the end of the model

        # Sequence in a block could be as follows
        # Convolution Layer -> ReLU -> Batch Normalization -> Max Pooling -> Dropout

        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            # nn.MaxPool2d(2, 2),
            nn.Dropout(0.2),
        )

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(16, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            # nn.Dropout(0.1),
            nn.Conv2d(16, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            # nn.Dropout(0.1),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.2),
        )

        # Block 3
        self.block3 = nn.Sequential(
            nn.Conv2d(16, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.2),
        )

        # Gap layer
        self.block4 = nn.Sequential(
            # nn.AdaptiveAvgPool2d(1),
            nn.AvgPool2d(kernel_size=5),
            nn.Flatten(),
            nn.Linear(16, 10),
        )

    def forward(self, x):
        x = self.block1(x)
        # print(x.shape)

        x = self.block2(x)
        # print(x.shape)

        x = self.block3(x)
        # print(x.shape)

        x = self.block4(x)
        # print(x.shape)

        return F.log_softmax(x)

In [ ]:
# Check if GPU is available and set device accordingly
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Move model to device and print summary given image size
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
            Conv2d-4           [-1, 16, 24, 24]           2,320
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
            Conv2d-7           [-1, 16, 24, 24]           2,320
              ReLU-8           [-1, 16, 24, 24]               0
       BatchNorm2d-9           [-1, 16, 24, 24]              32
          Dropout-10           [-1, 16, 24, 24]               0
           Conv2d-11           [-1, 16, 22, 22]           2,320
             ReLU-12           [-1, 16, 22, 22]               0
      BatchNorm2d-13           [-1, 16, 22, 22]              32
           Conv2d-14           [-1, 16,

<ipython-input-3-eb8db758dc79>:96: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Set seed for consistent results
torch.manual_seed(297)
# Set batch size
batch_size = 128

# Set kwargs for GPU if available
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

# Load train and test data and apply required transformations
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data",
        train=False,
        transform=transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

100%|██████████| 9912422/9912422 [00:00<00:00, 94599256.97it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 112266630.05it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26332710.40it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 19619494.10it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
# Train the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc=f"loss={loss.item()} batch_id={batch_idx}")


# Test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    # print test accuracy and loss
    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

In [ ]:
# Send model to device
model = Net().to(device)
# Set optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Specify the number of epochs
num_epochs = 20

# Train and test the model for specified number of epochs
for epoch in range(1, num_epochs + 1):
    if epoch == 1:
        print(f"\n\nBatch size: {batch_size}, Total epochs: {num_epochs}\n\n")

    print(f"\nEpoch {epoch}:\n")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)



Batch size: 128, Total epochs: 20



Epoch 1:



  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-3-eb8db758dc79>:96: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.07905068248510361 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.19it/s]



Test set: Average loss: 0.1074, Accuracy: 9698/10000 (96.98%)


Epoch 2:



loss=0.09110289067029953 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.33it/s]



Test set: Average loss: 0.0396, Accuracy: 9883/10000 (98.83%)


Epoch 3:



...


Test set: Average loss: 0.0169, Accuracy: 9948/10000 (99.48%)


Epoch 19:



loss=0.042367905378341675 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.90it/s]



Test set: Average loss: 0.0188, Accuracy: 9939/10000 (99.39%)


Epoch 20:



loss=0.004232310224324465 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.23it/s]



Test set: Average loss: 0.0196, Accuracy: 9937/10000 (99.37%)

